# **Manajemen & Berita Tf idf Dan Cbow**

## **Load Data**

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time
from google.colab import drive
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
pta_trunojoyo_path = "/content/drive/MyDrive/PPW/output/pta_manajemen.csv"
pta_trunojoyo = pd.read_csv(pta_trunojoyo_path)

pta_trunojoyo


,id,penulis,judul,abstrak_id,pembimbing_pertama,pembimbing_kedua,prodi
0,80211100070,SATIYAH,PENGARUH FAKTOR-FAKTOR PELATIHAN DAN PENGEMBAN...,"ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...","Dra. Hj. S. Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST.SE,M.MT",Manajemen
1,90211200001,Faishal,ANALISIS PERSEPSI BRAND ASSOCIATION MENURUT PE...,Tujuan penelitian ini adalah untuk mengetahui ...,Nurita Andriani,Yustina Chrismardani,Manajemen
2,80211100050,Wahyu Kurniawan,PENGARUH GAYA KEPEMIMPINAN DEMOKRATIK TERHADAP...,NaN,"Dr. Dra. Hj. Iriani Ismail, MM","Dra. Hj. S. Anugrahini Irawati, MM",Manajemen
3,100211200002,Muhammad Zakaria Utomo,Pengukuran Website Quality Pada Situs Sistem A...,Aplikasi nyata pemanfaatan teknologi informasi...,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Manajemen
4,80211100044,Hendri Wahyudi Prayitno,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Abstrak\r\nPenelitian ini menggunakan metode k...,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Manajemen
...,...,...,...,...,...,...,...
1026,160211100071,Husnul Hotimah,Analisis Cost Volume Profit Untuk Menentukan T...,ABSTRAK\nPenelitian ini bertujuan untuk menget...,"Hj. Evaliati Amaniyah, S.E., M.S.M.",NaN,Manajemen
1027,160211100291,Uswatun Hasanah,Pengaruh Pelatihan Dan Kompensasi Terhadap Pro...,"ABSTRAK\nUswatun Hasanah, 160211100291, Pengar...","Dr. Raden Mas Mochammad Wispandono S.E ., MS",NaN,Manajemen
1028,160211100064,ACH FATHONI,PERAN SERVICE PERFORMANCE DAN CLIMATE ORGANIZA...,ABSTRAK\nTujuan dari penelitian ini adalah unt...,"YUDHI PRASETYA MADA, S.E., M.M.",NaN,Manajemen
1029,160211100030,INTAN YULLIA NINGSIH,BAURAN PROMOSI PADA DEALER YAMAHA TRETAN MOTOR...,ABSTRAK\nPenelitian ini bertujuan: (1) Untuk m...,"DR. MOHAMMAD ARIEF, S.E., M.M.",NaN,Manajemen


## **Mengecek Nilai yang null dan memebersihkannya**


In [7]:
# Cek jumlah NaN di tiap kolom
print(pta_trunojoyo.isna().sum())


id                     0
penulis                2
judul                  0
abstrak_id             5
pembimbing_pertama     3
pembimbing_kedua      94
prodi                  0
dtype: int64


In [8]:
# Hapus baris yang abstrak_en kosong
df = pta_trunojoyo.dropna(subset=['abstrak_id']).reset_index(drop=True)

# Cek lagi apakah masih ada yang kosong
print(df['abstrak_id'].isna().sum())


0


In [9]:
df

,id,penulis,judul,abstrak_id,pembimbing_pertama,pembimbing_kedua,prodi
0,80211100070,SATIYAH,PENGARUH FAKTOR-FAKTOR PELATIHAN DAN PENGEMBAN...,"ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...","Dra. Hj. S. Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST.SE,M.MT",Manajemen
1,90211200001,Faishal,ANALISIS PERSEPSI BRAND ASSOCIATION MENURUT PE...,Tujuan penelitian ini adalah untuk mengetahui ...,Nurita Andriani,Yustina Chrismardani,Manajemen
2,100211200002,Muhammad Zakaria Utomo,Pengukuran Website Quality Pada Situs Sistem A...,Aplikasi nyata pemanfaatan teknologi informasi...,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Manajemen
3,80211100044,Hendri Wahyudi Prayitno,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Abstrak\r\nPenelitian ini menggunakan metode k...,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Manajemen
4,80211100119,Aththaariq,PENGARUH KOMPETENSI DOSEN TERHADAP KINERJA DOS...,"Abstrak\r\n\r\nAththaariq, Pengaruh Kompetensi...","Dr.RM Moch Wispandono,.S.E,.MS","Dr. Muhammad Alkirom Wildan,S.E.,M.Si.",Manajemen
...,...,...,...,...,...,...,...
1021,160211100071,Husnul Hotimah,Analisis Cost Volume Profit Untuk Menentukan T...,ABSTRAK\nPenelitian ini bertujuan untuk menget...,"Hj. Evaliati Amaniyah, S.E., M.S.M.",NaN,Manajemen
1022,160211100291,Uswatun Hasanah,Pengaruh Pelatihan Dan Kompensasi Terhadap Pro...,"ABSTRAK\nUswatun Hasanah, 160211100291, Pengar...","Dr. Raden Mas Mochammad Wispandono S.E ., MS",NaN,Manajemen
1023,160211100064,ACH FATHONI,PERAN SERVICE PERFORMANCE DAN CLIMATE ORGANIZA...,ABSTRAK\nTujuan dari penelitian ini adalah unt...,"YUDHI PRASETYA MADA, S.E., M.M.",NaN,Manajemen
1024,160211100030,INTAN YULLIA NINGSIH,BAURAN PROMOSI PADA DEALER YAMAHA TRETAN MOTOR...,ABSTRAK\nPenelitian ini bertujuan: (1) Untuk m...,"DR. MOHAMMAD ARIEF, S.E., M.M.",NaN,Manajemen


## **Cleaning Data**

Kode tersebut melakukan pembersihan teks pada kolom abstrak_indo dengan mengubah semua huruf menjadi huruf kecil agar konsisten dan tidak membedakan antara huruf besar dan kecil. Selain itu, kode ini menghapus semua angka, tanda baca, dan simbol sehingga hanya tersisa huruf dan spasi, yang bertujuan menghilangkan karakter yang tidak relevan atau mengganggu proses analisis teks. Terakhir, pembersihan juga menghapus spasi berlebihan, termasuk spasi ganda atau spasi di awal dan akhir kalimat, agar teks menjadi lebih rapi dan mudah diproses. Proses ini penting untuk menyederhanakan dan menormalkan data teks sehingga siap digunakan dalam tahap analisis lebih lanjut seperti tokenisasi, stemming, atau klasifikasi.

In [10]:

import pandas as pd
import re

# Fungsi cleansing
def cleansing(text):
    text = str(text).lower()                         # ubah ke huruf kecil
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)        # hilangkan angka/simbol
    text = re.sub(r'\s+', ' ', text).strip()        # hilangkan spasi berlebih
    return text

# Terapkan cleansing
df['cleaned'] = df['abstrak_id'].apply(cleansing)

# Simpan ke CSV
df.to_csv('/content/drive/MyDrive/PPW/output/manajemen_abstrak_cleaned.csv', index=False)
print("Data berhasil disimpan ke CSV!\n")

# Tampilkan beberapa baris pertama
df[['abstrak_id','cleaned']].head()

Data berhasil disimpan ke CSV!



,abstrak_id,cleaned
0,"ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...",abstrak satiyah pengaruh faktor faktor pelatih...
1,Tujuan penelitian ini adalah untuk mengetahui ...,tujuan penelitian ini adalah untuk mengetahui ...
2,Aplikasi nyata pemanfaatan teknologi informasi...,aplikasi nyata pemanfaatan teknologi informasi...
3,Abstrak\r\nPenelitian ini menggunakan metode k...,abstrak penelitian ini menggunakan metode kuan...
4,"Abstrak\r\n\r\nAththaariq, Pengaruh Kompetensi...",abstrak aththaariq pengaruh kompetensi dosen t...


In [11]:
!pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 38.6 MB/s eta 0:00:00


## **Stopword Removal**
Kode ini digunakan untuk menghapus kata-kata umum (stopwords) dalam bahasa Indonesia dari teks yang sudah dibersihkan di kolom cleaned. Dengan menggunakan library Sastrawi, dibuat objek stopword remover yang akan menghilangkan kata-kata seperti “dan”, “di”, “yang”, dan kata umum lain yang biasanya tidak membawa makna penting dalam analisis teks.

Setelah proses penghilangan stopwords selesai, hasilnya disimpan dalam kolom baru bernama no_stopwords. Kemudian, data yang sudah diproses ini disimpan kembali ke file CSV baru manajemen_abstrak_no_stopwords.csv. Langkah ini penting agar teks menjadi lebih fokus pada kata-kata bermakna dan memudahkan analisis selanjutnya seperti klasifikasi atau clustering.

In [12]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.2 MB/s eta 0:00:00


In [13]:
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Buat stopword remover
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# Terapkan stopword removal pada kolom 'cleaned'
df['no_stopwords'] = df['cleaned'].apply(stopword.remove)

# Simpan hasil ke CSV
df.to_csv('/content/drive/MyDrive/PPW/output/manajemen_abstrak_no_stopwords.csv', index=False)
print("Data berhasil disimpan ke CSV!\n")

# Tampilkan beberapa baris pertama
df[['cleaned','no_stopwords']].head()


Data berhasil disimpan ke CSV!



,cleaned,no_stopwords
0,abstrak satiyah pengaruh faktor faktor pelatih...,abstrak satiyah pengaruh faktor faktor pelatih...
1,tujuan penelitian ini adalah untuk mengetahui ...,tujuan penelitian mengetahui persepsi brand as...
2,aplikasi nyata pemanfaatan teknologi informasi...,aplikasi nyata pemanfaatan teknologi informasi...
3,abstrak penelitian ini menggunakan metode kuan...,abstrak penelitian menggunakan metode kuantita...
4,abstrak aththaariq pengaruh kompetensi dosen t...,abstrak aththaariq pengaruh kompetensi dosen k...


## **Stemming**
Kode ini melakukan proses stemming pada teks di kolom no_stopwords menggunakan library Sastrawi, yaitu mengubah kata-kata yang memiliki imbuhan atau variasi menjadi bentuk dasar atau kata dasarnya. Hasil stemming ini disimpan dalam kolom baru stemmed dan kemudian disimpan ke file CSV baru untuk digunakan dalam analisis selanjutnya. Proses stemming penting untuk menyederhanakan variasi kata sehingga model atau analisis teks dapat mengenali kata-kata dengan makna yang sama secara konsisten.

In [14]:
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Buat stemmer
stemmer = StemmerFactory().create_stemmer()

# Terapkan stemming pada kolom 'no_stopwords'
df['stemmed'] = df['no_stopwords'].apply(stemmer.stem)

# Simpan hasil ke CSV
df.to_csv('/content/drive/MyDrive/PPW/output/manajemen_abstrak_stemmed.csv', index=False)
print("Data berhasil disimpan ke CSV!\n")

# Tampilkan beberapa baris pertama
df[['no_stopwords','stemmed']].head()


Data berhasil disimpan ke CSV!



,no_stopwords,stemmed
0,abstrak satiyah pengaruh faktor faktor pelatih...,abstrak satiyah pengaruh faktor faktor latih k...
1,tujuan penelitian mengetahui persepsi brand as...,tuju teliti tahu persepsi brand association la...
2,aplikasi nyata pemanfaatan teknologi informasi...,aplikasi nyata manfaat teknologi informasi kom...
3,abstrak penelitian menggunakan metode kuantita...,abstrak teliti guna metode kuantitatif tekan u...
4,abstrak aththaariq pengaruh kompetensi dosen k...,abstrak aththaariq pengaruh kompetensi dosen k...


## **Tokenisasi**
Kode ini melakukan tokenisasi pada teks yang sudah melalui proses stemming di kolom stemmed. Tokenisasi adalah proses memecah kalimat atau teks menjadi potongan-potongan kata (token) yang lebih kecil, biasanya berdasarkan spasi. Fungsi tokenize yang sederhana di sini memecah setiap kalimat menjadi daftar kata-kata dengan menggunakan metode split(). Hasil tokenisasi disimpan dalam kolom baru tokens, yang berisi daftar kata untuk tiap baris teks. Setelah itu, data lengkap dengan token disimpan ke file CSV baru manajemen_abstrak_tokens.csv

In [15]:
import pandas as pd

# Fungsi tokenisasi sederhana
def tokenize(text):
    return text.split()

# Terapkan tokenisasi pada kolom 'stemmed'
df['tokens'] = df['stemmed'].apply(tokenize)

# Simpan hasil ke CSV
df.to_csv('/content/drive/MyDrive/PPW/output/manajemen_abstrak_tokens.csv', index=False)
print("Data berhasil disimpan ke CSV!\n")

# Tampilkan beberapa baris pertama
df[['stemmed','tokens']].head()


Data berhasil disimpan ke CSV!



,stemmed,tokens
0,abstrak satiyah pengaruh faktor faktor latih k...,"[abstrak, satiyah, pengaruh, faktor, faktor, l..."
1,tuju teliti tahu persepsi brand association la...,"[tuju, teliti, tahu, persepsi, brand, associat..."
2,aplikasi nyata manfaat teknologi informasi kom...,"[aplikasi, nyata, manfaat, teknologi, informas..."
3,abstrak teliti guna metode kuantitatif tekan u...,"[abstrak, teliti, guna, metode, kuantitatif, t..."
4,abstrak aththaariq pengaruh kompetensi dosen k...,"[abstrak, aththaariq, pengaruh, kompetensi, do..."


## **TF-IDF**

### **Ambil Corpus**

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Ambil corpus dari kolom stemmed
corpus = df['stemmed'].astype(str).tolist()

print("Jumlah dokumen dalam corpus:", len(corpus))
print("Contoh dokumen:", corpus[0][:200])


Jumlah dokumen dalam corpus: 1026
Contoh dokumen: abstrak satiyah pengaruh faktor faktor latih kembang produktivitas kerja dinas laut ikan bangkal bawah bimbing dra hj s anugrahini irawati mm helm buyung aulia s st se m mt upaya tingkat produktivitas


In [17]:
# Inisialisasi TF-IDF
vectorizer = TfidfVectorizer()

# Fit dan transform
X_tfidf = vectorizer.fit_transform(corpus)

print("Shape TF-IDF:", X_tfidf.shape)  # (jumlah dokumen, jumlah kata unik)


Shape TF-IDF: (1026, 6056)


In [21]:
# Ubah ke DataFrame biar mudah dibaca
tfidf_df = pd.DataFrame(
    X_tfidf.toarray(),
    columns=vectorizer.get_feature_names_out()
)

# Simpan hasil ke CSV
tfidf_df.to_csv("/content/drive/MyDrive/PPW/output/tfidf_hasil2.csv", index=False)
print("Data berhasil disimpan ke CSV!\n")

# Tampilkan 5 baris pertama
tfidf_df.head()


Data berhasil disimpan ke CSV!



,aaa,aaaamanahsyariah,aar,abadi,abai,abalisis,abc,abcs,abd,abdul,...,zscore,zte,zuhri,zuhruf,zulfi,zulia,zuliana,zulkifli,zulpah,zyn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Tampilkan 10 kata dengan bobot TF-IDF tertinggi di dokumen pertama
print(tfidf_df.iloc[0].sort_values(ascending=False).head(10))

produktivitas    0.411053
laut             0.331714
ikan             0.331714
latih            0.328843
dinas            0.234956
kembang          0.223807
pegawai          0.223024
faktor           0.222064
seleksi          0.165857
kerja            0.158086
Name: 0, dtype: float64


## **CBOW**

In [23]:
!pip install gensim

In [24]:
from gensim.models import Word2Vec

corpus = []
for col in df['stemmed']:
    word_list = col.split(" ")
    corpus.append(word_list)

print(corpus[0][:20])

['abstrak', 'satiyah', 'pengaruh', 'faktor', 'faktor', 'latih', 'kembang', 'produktivitas', 'kerja', 'dinas', 'laut', 'ikan', 'bangkal', 'bawah', 'bimbing', 'dra', 'hj', 's', 'anugrahini', 'irawati']


In [25]:
# Training Word2Vec
model = Word2Vec(
    corpus,
    vector_size=56,   # ukuran embedding
    window=5,
    min_count=1,
    sg=0   # CBOW
)


In [26]:
import numpy as np

class MeanEmbeddingVectorizer:
    def __init__(self, model):
        self.model = model
        self.vector_size = model.vector_size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.model.wv[word] for word in words if word in self.model.wv]
                    or [np.zeros(self.vector_size)], axis=0)
            for words in X
        ])

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)


In [27]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(model)
mean_embedded = mean_embedding_vectorizer.fit_transform(corpus)

# simpan ke DataFrame
df['array'] = list(mean_embedded)

# cek panjang embedding
df['embedding_length'] = df['array'].str.len()
print(df[['stemmed','embedding_length']].head())


                                             stemmed  embedding_length
0  abstrak satiyah pengaruh faktor faktor latih k...                56
1  tuju teliti tahu persepsi brand association la...                56
2  aplikasi nyata manfaat teknologi informasi kom...                56
3  abstrak teliti guna metode kuantitatif tekan u...                56
4  abstrak aththaariq pengaruh kompetensi dosen k...                56


In [28]:
num_features = len(df['array'].iloc[0])  # asumsi semua list punya panjang sama
columns = [f'f{i+1}' for i in range(num_features)]

# Inisialisasi dictionary untuk menampung data per kolom
data_dict = {col: [] for col in columns}

# Looping setiap baris di kolom 'embedding'
for embedding_list in df['array']:
    for i, value in enumerate(embedding_list):
        data_dict[f'f{i+1}'].append(value)

# Buat DataFrame dari dictionary
embedding_df = pd.DataFrame(data_dict)

print(embedding_df)

            f1        f2        f3        f4        f5        f6        f7  \
0    -0.604110  0.601810  0.002295  0.079372  0.459393 -0.667764  0.647536   
1    -0.407547  0.889265  0.370033 -0.146664  0.072259 -0.723037  0.532797   
2    -0.393729  0.661485  0.165723  0.022622  0.297205 -0.600596  0.320031   
3    -0.527969  1.225806  0.151130  0.318914  0.778568 -0.336958  1.332707   
4    -0.493933  0.959758  0.263732  0.156412  0.414333 -0.523047  0.879891   
...        ...       ...       ...       ...       ...       ...       ...   
1021 -0.091734  0.876863 -0.062520  0.216218  0.395602 -0.091359  0.537025   
1022 -0.473220  1.031010 -0.181029  0.512152  1.295863 -0.163428  1.406860   
1023 -0.356133  1.139736  0.363296  0.280302  0.649547 -0.355307  1.045351   
1024 -0.611105  1.254250  0.393739 -0.199842 -0.034729 -0.644291  0.387719   
1025 -0.608394  0.550723  0.000963  0.105905  0.657768 -0.669823  0.613910   

            f8        f9       f10  ...       f47       f48    

In [29]:
embedding_df['abstrak_id'] = df['abstrak_id'].values

In [30]:
embedding_df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f48,f49,f50,f51,f52,f53,f54,f55,f56,abstrak_id
0,-0.604110,0.601810,0.002295,0.079372,0.459393,-0.667764,0.647536,-0.372226,-0.216750,-0.329751,...,1.106180,0.008137,0.566406,-0.049424,1.025979,-0.252113,-0.378402,0.284748,-0.274221,"ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel..."
1,-0.407547,0.889265,0.370033,-0.146664,0.072259,-0.723037,0.532797,-0.278238,-0.065625,-0.275656,...,0.431312,0.024516,0.335085,-0.224213,0.772517,-0.470658,-0.099382,0.391731,-0.044709,Tujuan penelitian ini adalah untuk mengetahui ...
2,-0.393729,0.661485,0.165723,0.022622,0.297205,-0.600596,0.320031,-0.265646,-0.081141,-0.262838,...,0.526340,-0.002028,0.296957,-0.229636,0.766397,-0.267639,-0.242218,0.464620,-0.064802,Aplikasi nyata pemanfaatan teknologi informasi...
3,-0.527969,1.225806,0.151130,0.318914,0.778568,-0.336958,1.332707,-0.299899,-0.102961,-0.516676,...,1.379737,-0.213524,0.722797,-0.066406,1.515525,-0.313497,-0.500740,0.254505,-0.250542,Abstrak\r\nPenelitian ini menggunakan metode k...
4,-0.493933,0.959758,0.263732,0.156412,0.414333,-0.523047,0.879891,-0.463371,-0.252314,-0.484654,...,0.967058,-0.198286,0.491361,-0.179367,1.185256,-0.313823,-0.357226,0.316788,-0.278904,"Abstrak\r\n\r\nAththaariq, Pengaruh Kompetensi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,-0.091734,0.876863,-0.062520,0.216218,0.395602,-0.091359,0.537025,-0.664627,-0.109089,-0.482788,...,0.548367,-0.306723,0.134827,0.092991,0.935386,-0.252430,-0.023406,0.091227,0.256946,ABSTRAK\nPenelitian ini bertujuan untuk menget...
1022,-0.473220,1.031010,-0.181029,0.512152,1.295863,-0.163428,1.406860,-0.758672,-0.244727,-0.690512,...,1.632496,-0.322082,0.567944,0.274575,1.761353,-0.018669,-0.597444,0.165465,-0.350276,"ABSTRAK\nUswatun Hasanah, 160211100291, Pengar..."
1023,-0.356133,1.139736,0.363296,0.280302,0.649547,-0.355307,1.045351,-0.427418,-0.101017,-0.603227,...,0.834670,-0.180596,0.423346,-0.092497,1.301476,-0.315845,-0.255158,0.272079,0.052127,ABSTRAK\nTujuan dari penelitian ini adalah unt...
1024,-0.611105,1.254250,0.393739,-0.199842,-0.034729,-0.644291,0.387719,-0.322859,0.106902,-0.478996,...,1.262082,-0.170142,0.342091,-0.150756,0.899229,-0.255359,-0.366396,0.499736,-0.280610,ABSTRAK\nPenelitian ini bertujuan: (1) Untuk m...


In [31]:
embedding_df.shape

(1026, 57)

# **BERITA TF IDF & CBOW**

## **Load Data**

In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time
from google.colab import drive
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [33]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
berita_cnn_path = "/content/drive/MyDrive/PPW/output/cnnindonesia_berita4.csv"
berita_cnn = pd.read_csv(berita_cnn_path)

berita_cnn

,id_berita,judul_berita,isi_berita,kategori_berita
0,1,"Dorong Suporter, Andre Onana Dilempar BotolOla...",Kiper Manchester UnitedAndre Onanadilempar bot...,olahraga
1,2,Militer Nepal Batasi Aktivitas Pascademo Gulin...,Militer Nepalmengambil kembali kendali atas Ka...,internasional
2,3,Dua Helikopter Disiagakan Selama MotoGP Mandal...,Basarnas akan menyiapkan dan menyiagakan dua h...,olahraga
3,4,"Lari, Rahasia Hidup Sehat untuk Tingkatkan Pro...",Tingginya tuntutan pekerjaan di era yang serba...,gaya-hidup
4,5,Lokasi Titik Jatuh Helikopter di Mimika Ditemu...,Kepala Badan Nasional Pencarian dan Pertolonga...,nasional
...,...,...,...,...
995,996,"7 FOTOFOTO: Rusia Bombardir Ukraina, Luncurkan...",NaN,internasional
996,997,"Hasil MotoGP Catalunya: Alex Jadi Pemenang, Ka...",Pembalap Gresini Racing Alex Marquez memenangk...,olahraga
997,998,"Prabowo: Tuntutan 17+8 Sebagian Masuk Akal, Se...",Presiden RIPrabowo Subiantomenyatakan sebagian...,nasional
998,999,00:57VIDEO: PM Jepang Shigeru Ishiba Umumkan M...,"Perdana Menteri Jepang, Shigeru Ishiba, mengun...",internasional


In [36]:
# Melihat daftar kategori unik
print(berita_cnn['kategori_berita'].unique())

# Menghitung jumlah kategori unik
print("Jumlah kategori:", berita_cnn['kategori_berita'].nunique())

# Kalau mau lihat jumlah data per kategori
print(berita_cnn['kategori_berita'].value_counts())

['olahraga' 'internasional' 'gaya-hidup' 'nasional' 'tv' 'ekonomi'
 'hiburan' 'otomotif' 'edukasi' 'teknologi']
Jumlah kategori: 10
kategori_berita
nasional         259
ekonomi          169
olahraga         167
internasional    119
tv                76
gaya-hidup        67
hiburan           51
edukasi           35
teknologi         33
otomotif          24
Name: count, dtype: int64


## **Mengecek Nilai yang null dan memebersihkannya**


In [37]:
# Cek jumlah NaN di tiap kolom
print(berita_cnn.isna().sum())

id_berita           0
judul_berita        0
isi_berita         41
kategori_berita     0
dtype: int64


In [38]:
# Hapus baris yang abstrak_en kosong
data = berita_cnn.dropna(subset=['isi_berita']).reset_index(drop=True)

# Cek lagi apakah masih ada yang kosong
print(data['isi_berita'].isna().sum())


0


In [39]:
data

,id_berita,judul_berita,isi_berita,kategori_berita
0,1,"Dorong Suporter, Andre Onana Dilempar BotolOla...",Kiper Manchester UnitedAndre Onanadilempar bot...,olahraga
1,2,Militer Nepal Batasi Aktivitas Pascademo Gulin...,Militer Nepalmengambil kembali kendali atas Ka...,internasional
2,3,Dua Helikopter Disiagakan Selama MotoGP Mandal...,Basarnas akan menyiapkan dan menyiagakan dua h...,olahraga
3,4,"Lari, Rahasia Hidup Sehat untuk Tingkatkan Pro...",Tingginya tuntutan pekerjaan di era yang serba...,gaya-hidup
4,5,Lokasi Titik Jatuh Helikopter di Mimika Ditemu...,Kepala Badan Nasional Pencarian dan Pertolonga...,nasional
...,...,...,...,...
954,995,"Pecahkan Rekor, Pria 102 Tahun Berhasil Capai ...","Kokichi Akuzawa, pria asalJepang, berhasil pec...",gaya-hidup
955,997,"Hasil MotoGP Catalunya: Alex Jadi Pemenang, Ka...",Pembalap Gresini Racing Alex Marquez memenangk...,olahraga
956,998,"Prabowo: Tuntutan 17+8 Sebagian Masuk Akal, Se...",Presiden RIPrabowo Subiantomenyatakan sebagian...,nasional
957,999,00:57VIDEO: PM Jepang Shigeru Ishiba Umumkan M...,"Perdana Menteri Jepang, Shigeru Ishiba, mengun...",internasional


## **Cleaning Data**

In [40]:
import pandas as pd
import re

# Fungsi cleansing
def cleansing(text):
    text = str(text).lower()                         # ubah ke huruf kecil
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)        # hilangkan angka/simbol
    text = re.sub(r'\s+', ' ', text).strip()        # hilangkan spasi berlebih
    return text

# Terapkan cleansing
data['cleaned'] = data['isi_berita'].apply(cleansing)

# Simpan ke CSV
data.to_csv('/content/drive/MyDrive/PPW/output/isi_berita_cleaned.csv', index=False)
print("Data berhasil disimpan ke CSV!\n")

# Tampilkan beberapa baris pertama
data[['isi_berita','cleaned']].head()

Data berhasil disimpan ke CSV!



,isi_berita,cleaned
0,Kiper Manchester UnitedAndre Onanadilempar bot...,kiper manchester unitedandre onanadilempar bot...
1,Militer Nepalmengambil kembali kendali atas Ka...,militer nepalmengambil kembali kendali atas ka...
2,Basarnas akan menyiapkan dan menyiagakan dua h...,basarnas akan menyiapkan dan menyiagakan dua h...
3,Tingginya tuntutan pekerjaan di era yang serba...,tingginya tuntutan pekerjaan di era yang serba...
4,Kepala Badan Nasional Pencarian dan Pertolonga...,kepala badan nasional pencarian dan pertolonga...


In [41]:
!pip install pyspellchecker

## **Stopword Removal**

In [42]:
!pip install Sastrawi

In [43]:
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Buat stopword remover
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# Terapkan stopword removal pada kolom 'cleaned'
data['no_stopwords'] = data['cleaned'].apply(stopword.remove)

# Simpan hasil ke CSV
data.to_csv('/content/drive/MyDrive/PPW/output/berita_cnn_no_stopwords.csv', index=False)
print("Data berhasil disimpan ke CSV!\n")

# Tampilkan beberapa baris pertama
data[['cleaned','no_stopwords']].head()


Data berhasil disimpan ke CSV!



,cleaned,no_stopwords
0,kiper manchester unitedandre onanadilempar bot...,kiper manchester unitedandre onanadilempar bot...
1,militer nepalmengambil kembali kendali atas ka...,militer nepalmengambil kendali atas kathmandu ...
2,basarnas akan menyiapkan dan menyiagakan dua h...,basarnas menyiapkan menyiagakan helikopter eva...
3,tingginya tuntutan pekerjaan di era yang serba...,tingginya tuntutan pekerjaan era serba cepat s...
4,kepala badan nasional pencarian dan pertolonga...,kepala badan nasional pencarian pertolongan ba...


## **Stemming**

In [44]:
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Buat stemmer
stemmer = StemmerFactory().create_stemmer()

# Terapkan stemming pada kolom 'no_stopwords'
data['stemmed'] = data['no_stopwords'].apply(stemmer.stem)

# Simpan hasil ke CSV
data.to_csv('/content/drive/MyDrive/PPW/output/berita_cnn_stemmed.csv', index=False)
print("Data berhasil disimpan ke CSV!\n")

# Tampilkan beberapa baris pertama
data[['no_stopwords','stemmed']].head()


Data berhasil disimpan ke CSV!



,no_stopwords,stemmed
0,kiper manchester unitedandre onanadilempar bot...,kiper manchester unitedandre onanadilempar bot...
1,militer nepalmengambil kendali atas kathmandu ...,militer nepalmengambil kendali atas kathmandu ...
2,basarnas menyiapkan menyiagakan helikopter eva...,basarnas siap siaga helikopter evakuasi medis ...
3,tingginya tuntutan pekerjaan era serba cepat s...,tinggi tuntut kerja era serba cepat seringkali...
4,kepala badan nasional pencarian pertolongan ba...,kepala badan nasional cari tolong basarnas mar...


## **Tokenisasi**

In [45]:
import pandas as pd

# Fungsi tokenisasi sederhana
def tokenize(text):
    return text.split()

# Terapkan tokenisasi pada kolom 'stemmed'
data['tokens'] = data['stemmed'].apply(tokenize)

# Simpan hasil ke CSV
data.to_csv('/content/drive/MyDrive/PPW/output/berita_cnn_tokens.csv', index=False)
print("Data berhasil disimpan ke CSV!\n")

# Tampilkan beberapa baris pertama
data[['stemmed','tokens']].head()


Data berhasil disimpan ke CSV!



,stemmed,tokens
0,kiper manchester unitedandre onanadilempar bot...,"[kiper, manchester, unitedandre, onanadilempar..."
1,militer nepalmengambil kendali atas kathmandu ...,"[militer, nepalmengambil, kendali, atas, kathm..."
2,basarnas siap siaga helikopter evakuasi medis ...,"[basarnas, siap, siaga, helikopter, evakuasi, ..."
3,tinggi tuntut kerja era serba cepat seringkali...,"[tinggi, tuntut, kerja, era, serba, cepat, ser..."
4,kepala badan nasional cari tolong basarnas mar...,"[kepala, badan, nasional, cari, tolong, basarn..."


## **TF-IDF**

### **Ambil Corpus**

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Ambil corpus dari kolom stemmed
corpus = data['stemmed'].astype(str).tolist()

print("Jumlah dokumen dalam corpus:", len(corpus))
print("Contoh dokumen:", corpus[0][:200])

Jumlah dokumen dalam corpus: 959
Contoh dokumen: kiper manchester unitedandre onanadilempar botol orang suporter tolak swafoto setelahkamerunkalah cape verde kualifikasi piala dunia zona afrika kamerun gagal tempel ketat cape verde saing grup d kual


In [47]:
# Inisialisasi TF-IDF
vectorizer = TfidfVectorizer()

# Fit dan transform
X_tfidf = vectorizer.fit_transform(corpus)

print("Shape TF-IDF:", X_tfidf.shape)  # (jumlah dokumen, jumlah kata unik)


Shape TF-IDF: (959, 14560)


In [49]:
# Ubah ke DataFrame biar mudah dibaca
tfidf_data = pd.DataFrame(
    X_tfidf.toarray(),
    columns=vectorizer.get_feature_names_out()
)

# Simpan hasil ke CSV
tfidf_data.to_csv("/content/drive/MyDrive/PPW/output/tfidf_hasilberita.csv", index=False)
print("Data berhasil disimpan ke CSV!\n")

# Tampilkan 5 baris pertama
tfidf_data.head()


Data berhasil disimpan ke CSV!



,aaa,aaat,aala,aalaa,aan,aasgaard,aati,aatin,aayush,aba,...,zudan,zuhri,zuhur,zulhas,zulismayanto,zulkifli,zulpadli,zulu,zuxin,zwiers
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# Tampilkan 10 kata dengan bobot TF-IDF tertinggi di dokumen pertama
print(tfidf_data.iloc[0].sort_values(ascending=False).head(10))

onana         0.636944
cape          0.273156
verde         0.261546
kamerun       0.231616
suporter      0.210987
livramento    0.173712
kiper         0.166562
botol         0.151524
gol           0.113749
swafoto       0.104618
Name: 0, dtype: float64


## **CBOW**

In [51]:
!pip install gensim


In [52]:
from gensim.models import Word2Vec

corpus = []
for col in data['stemmed']:
    word_list = col.split(" ")
    corpus.append(word_list)

# Tampilkan contoh dokumen pertama
print(corpus[0][:20])


['kiper', 'manchester', 'unitedandre', 'onanadilempar', 'botol', 'orang', 'suporter', 'tolak', 'swafoto', 'setelahkamerunkalah', 'cape', 'verde', 'kualifikasi', 'piala', 'dunia', 'zona', 'afrika', 'kamerun', 'gagal', 'tempel']


In [53]:
# Training Word2Vec
model = Word2Vec(
    corpus,
    vector_size=56,   # ukuran embedding
    window=5,
    min_count=1,
    sg=0   # CBOW
)

In [54]:
import numpy as np

class MeanEmbeddingVectorizer:
    def __init__(self, model):
        self.model = model
        self.vector_size = model.vector_size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.model.wv[word] for word in words if word in self.model.wv]
                    or [np.zeros(self.vector_size)], axis=0)
            for words in X
        ])

    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)


In [55]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(model)
mean_embedded = mean_embedding_vectorizer.fit_transform(corpus)

# simpan ke DataFrame
data['array'] = list(mean_embedded)

# cek panjang embedding
data['embedding_length'] = data['array'].str.len()
print(data[['stemmed','embedding_length']].head())


                                             stemmed  embedding_length
0  kiper manchester unitedandre onanadilempar bot...                56
1  militer nepalmengambil kendali atas kathmandu ...                56
2  basarnas siap siaga helikopter evakuasi medis ...                56
3  tinggi tuntut kerja era serba cepat seringkali...                56
4  kepala badan nasional cari tolong basarnas mar...                56


In [56]:
num_features = len(data['array'].iloc[0])  # asumsi semua list punya panjang sama
columns = [f'f{i+1}' for i in range(num_features)]

# Inisialisasi dictionary untuk menampung data per kolom
data_dict = {col: [] for col in columns}

# Looping setiap baris di kolom 'embedding'
for embedding_list in data['array']:
    for i, value in enumerate(embedding_list):
        data_dict[f'f{i+1}'].append(value)

# Buat DataFrame dari dictionary
embedding_data = pd.DataFrame(data_dict)

print(embedding_data)

           f1        f2        f3        f4        f5        f6        f7  \
0   -0.043040  0.134250  0.066879  0.233033  0.055386 -0.437548  0.221575   
1   -0.325521  0.431703  0.022321  0.584841 -0.097776 -0.588445  0.462229   
2   -0.143116  0.268807  0.017777  0.380797 -0.067898 -0.513197  0.330286   
3   -0.131251  0.271456 -0.163505  0.467549 -0.210462 -0.494212  0.364010   
4   -0.177898  0.369744  0.079186  0.447666 -0.092382 -0.519650  0.333601   
..        ...       ...       ...       ...       ...       ...       ...   
954 -0.094745  0.314864 -0.001110  0.441133 -0.120885 -0.546837  0.391255   
955 -0.112289  0.185256  0.020707  0.246692  0.035334 -0.451383  0.262686   
956 -0.296370  0.399938  0.005976  0.571427 -0.161413 -0.597700  0.434890   
957 -0.478431  0.374611 -0.137550  0.665012 -0.246625 -0.732426  0.643991   
958 -0.241289  0.348417  0.009849  0.495396 -0.135717 -0.519798  0.397390   

           f8        f9       f10  ...       f47       f48       f49  \
0  

In [57]:
embedding_data['isi_berita'] = data['isi_berita'].values

In [59]:
embedding_data

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f48,f49,f50,f51,f52,f53,f54,f55,f56,isi_berita
0,-0.043040,0.134250,0.066879,0.233033,0.055386,-0.437548,0.221575,-0.918964,-0.287134,-0.353031,...,0.377630,0.146114,0.078160,0.140607,0.224496,-0.168424,0.073228,0.167214,-0.605861,Kiper Manchester UnitedAndre Onanadilempar bot...
1,-0.325521,0.431703,0.022321,0.584841,-0.097776,-0.588445,0.462229,-0.976615,-0.355678,-0.406668,...,0.734137,0.031134,0.023396,0.213832,0.578409,-0.213557,0.095616,0.173846,-0.375525,Militer Nepalmengambil kembali kendali atas Ka...
2,-0.143116,0.268807,0.017777,0.380797,-0.067898,-0.513197,0.330286,-0.912359,-0.259447,-0.304721,...,0.558514,0.039529,0.023762,0.169546,0.371774,-0.212224,0.030863,0.129127,-0.476111,Basarnas akan menyiapkan dan menyiagakan dua h...
3,-0.131251,0.271456,-0.163505,0.467549,-0.210462,-0.494212,0.364010,-1.002274,-0.158388,-0.100765,...,0.702717,-0.053635,-0.014211,0.172166,0.424065,-0.140338,-0.043736,0.179152,-0.518641,Tingginya tuntutan pekerjaan di era yang serba...
4,-0.177898,0.369744,0.079186,0.447666,-0.092382,-0.519650,0.333601,-0.950788,-0.275748,-0.389508,...,0.591208,0.045537,0.044788,0.179191,0.398439,-0.290352,0.024083,0.016249,-0.412154,Kepala Badan Nasional Pencarian dan Pertolonga...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,-0.094745,0.314864,-0.001110,0.441133,-0.120885,-0.546837,0.391255,-1.097195,-0.227641,-0.316176,...,0.626422,0.043689,0.128500,0.187164,0.451523,-0.234786,0.049748,0.071112,-0.594186,"Kokichi Akuzawa, pria asalJepang, berhasil pec..."
955,-0.112289,0.185256,0.020707,0.246692,0.035334,-0.451383,0.262686,-0.873857,-0.337155,-0.337484,...,0.342770,0.121936,0.125755,0.120497,0.278069,-0.160711,0.117937,0.136676,-0.559937,Pembalap Gresini Racing Alex Marquez memenangk...
956,-0.296370,0.399938,0.005976,0.571427,-0.161413,-0.597700,0.434890,-1.127918,-0.274221,-0.393258,...,0.809849,-0.028652,-0.123731,0.236669,0.573613,-0.275756,-0.012594,0.103562,-0.469039,Presiden RIPrabowo Subiantomenyatakan sebagian...
957,-0.478431,0.374611,-0.137550,0.665012,-0.246625,-0.732426,0.643991,-1.171741,-0.394689,-0.363042,...,0.906593,-0.098149,-0.170272,0.268817,0.787656,-0.165463,0.063955,0.354915,-0.467763,"Perdana Menteri Jepang, Shigeru Ishiba, mengun..."


In [60]:
embedding_data.shape

(959, 57)